In [2]:
import pandas as pd
from datetime import datetime

In [10]:
df = pd.read_csv('WeatherEvents_Jan2016-Dec2020.csv')

In [3]:
df.shape

(6274206, 13)

In [144]:
df.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [11]:
df[['StartTime(UTC)','EndTime(UTC)']] = df[['StartTime(UTC)','EndTime(UTC)']].apply(pd.to_datetime)

In [19]:
jan_df = df[ (df['EndTime(UTC)'] >= '2019-01-01') & (df['EndTime(UTC)'] <='2019-01-31') ]

In [25]:
test_df = jan_df.copy()

In [31]:
test_df['dest_city_name'] = test_df['City'] + ', ' + test_df['State']

In [52]:
test_df['Date'] = df['EndTime(UTC)'].apply(lambda x: datetime.date(x))

#### Test merging and functions on small data set

In [59]:
test2_df = test_df.copy()
test2_df = test2_df.sample(500)
check_df = test2_df.groupby(['dest_city_name','Date'], as_index=False).apply(lambda x: x.mode()['Type'][0])
check_df = check_df.set_axis(['dest_city_name', 'Date', 'Mode'],axis=1)
test_22_df = test2_df[['dest_city_name','Date']]
merge_test = test_22_df.merge(check_df, how='left', on=['dest_city_name','Date'])
merge_test.shape

#### Calculate Modes

In [130]:
# for type 

type_mode = test_df.groupby(['dest_city_name','Date'], as_index = False).apply(lambda x: x.mode()['Type'][0])
type_mode = type_mode.set_axis(['dest_city_name', 'Date', 'Type_Mode'],axis=1)
test_type_merge = test_df.merge(type_mode, how='left', on=['dest_city_name','Date'])

In [132]:
# for severity 

sev_mode = test_df.groupby(['dest_city_name','Date'], as_index = False).apply(lambda x: x.mode()['Severity'][0])
sev_mode = sev_mode.set_axis(['dest_city_name', 'Date', 'Severity_Mode'],axis=1)
test_df_final = test_type_merge.merge(sev_mode, how='left', on=['dest_city_name','Date'])

In [133]:
test_df_final.head(5)

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,dest_city_name,Date,Type_Mode,Severity_Mode
0,W-1274,Snow,Light,2018-12-31 21:53:00,2019-01-01 19:36:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"Saguache, CO",2019-01-01,Snow,Light
1,W-1275,Snow,Light,2019-01-02 01:54:00,2019-01-02 02:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"Saguache, CO",2019-01-02,Snow,Light
2,W-1276,Cold,Severe,2019-01-03 10:54:00,2019-01-03 11:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"Saguache, CO",2019-01-03,Cold,Severe
3,W-1277,Fog,Moderate,2019-01-03 12:56:00,2019-01-03 13:18:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"Saguache, CO",2019-01-03,Cold,Severe
4,W-1278,Cold,Severe,2019-01-03 13:18:00,2019-01-03 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"Saguache, CO",2019-01-03,Cold,Severe


In [ ]:
test_df_final.shape

In [163]:
df4 = test_df_final[['dest_city_name','Date','Type_Mode']]
df4 = df4.drop_duplicates()
df4 = df4.rename(columns={'dest_city_name':'origin_city_name', 'Date':'fl_date'})

In [148]:
df3 = test_df_final[['dest_city_name','Date','Type_Mode']]
df3 = df3.drop_duplicates()

,dest_city_name,Date,Type_Mode
0,"Saguache, CO",2019-01-01,Snow
1,"Saguache, CO",2019-01-02,Snow
2,"Saguache, CO",2019-01-03,Cold
6,"Saguache, CO",2019-01-07,Snow
8,"Saguache, CO",2019-01-11,Snow
...,...,...,...
121391,"Rock River, WY",2019-01-19,Storm
121394,"Rock River, WY",2019-01-23,Storm
121399,"Rock River, WY",2019-01-26,Storm
121403,"Rock River, WY",2019-01-27,Storm


In [153]:
df3.to_csv('mergetest.csv')